# Deep Learning - Exercise 5

This lecture is about advanced topics of the CNN such as Autoencoder for image reconstruction and Variatioanl autoencoder for generating images.

We use MNIST dataset for the tasks as it is quite easy to process.

[Open in Google colab](https://colab.research.google.com/github/rasvob/VSB-FEI-Deep-Learning-Exercises/blob/main/dl_05.ipynb)
[Download from Github](https://github.com/rasvob/VSB-FEI-Deep-Learning-Exercises/blob/main/dl_05.ipynb)

##### Remember to set **GPU** runtime in Colab!

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import pandas as pd
import matplotlib.pyplot as plt # plotting
import matplotlib.image as mpimg # images
import numpy as np #numpy
import tensorflow as tf
import tensorflow.keras as keras
import requests

from tensorflow.keras.layers import Activation
from tensorflow.keras.utils import get_custom_objects
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import VGG19

from sklearn.model_selection import train_test_split # split for validation sets
from sklearn.preprocessing import normalize # normalization of the matrix
from scipy.signal import convolve2d # convolutionof the 2D signals
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from PIL import Image
from io import BytesIO
from skimage.transform import resize
import cv2
import scipy

tf.version.VERSION

In [ ]:
def show_history(history):
    plt.figure()
    for key in history.history.keys():
        plt.plot(history.epoch, history.history[key], label=key)
    plt.legend()
    plt.tight_layout()

def show_example(train_x, train_y, class_names):
    plt.figure(figsize=(10,10))
    for i in range(25):
        plt.subplot(5,5,i+1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(train_x[i].reshape(28,28), cmap=plt.cm.binary)
        plt.xlabel(class_names[train_y[i]])
    plt.show()

## 🔎 What is an autoencoder? 🔎
* The autoencoder is a special type of neural network that is able to learn without the classes just from the input data 
    * It is equivalent to the feature extraction from the data
    * 🔎 Can you name any use-case for such model?
* What is the latent vector?
* How can we compare two images?

![Arch01](https://github.com/rasvob/VSB-FEI-Deep-Learning-Exercises/blob/main/images/dl_05_enc_arch.png?raw=true)

## The autoencoder uses binary-crossentropy as loss function
* How is the function used?


$BCE = -\frac{1}{N}\sum_{i=1}^{N}y_i*log(p(y_i))+(1-y_i*log(1-p(y_i)))$

* 💡 Beware the asymetry of the BCE function

* **During the backpropagation process BCE cares more about very bright (1) or very dark pixels (0), but puts less effort on optimizing middle tones (0.5).**

* Give the true value is 0.2, and autoencoder A predicts 0.1 while autoencoder B predicts 0.3. 
    * The loss for A would be: $−(0.2 * log_{10}(0.1) + (1−0.2) * log_{10}(1−0.2)) = 0.278$    
    * The loss for B would be: $−(0.2 * log_{10}(0.3) + (1−0.2) * log_{10}(1−0.3)) = 0.228$

* 📌 Check [this](https://towardsdatascience.com/understanding-binary-cross-entropy-log-loss-a-visual-explanation-a3ac6025181a) or [this](https://stats.stackexchange.com/questions/394582/why-is-binary-cross-entropy-or-log-loss-used-in-autoencoders-for-non-binary-da) for more details

# 🚀 Let's start!
* We will use *MNIST* dataset again for this task

In [ ]:
# mnist is the basic dataset for image classifaction
dataset = tf.keras.datasets.mnist

# data from any dataset are loaded using the load_Data function
(train_x, train_y), (test_x, test_y) = dataset.load_data()

train_x = train_x.reshape(*train_x.shape, 1)
test_x = test_x.reshape(*test_x.shape, 1)

train_x = train_x/255.0
test_x = test_x/255.0

train_x, valid_x, train_y, valid_y = train_test_split(train_x, train_y, test_size=0.2, random_state=42)

# the data are in the form of 28x28 pixes with values 0-255.
print('Train data shape: ', train_x.shape, train_y.shape)
print('Validation data shape: ', valid_x.shape, valid_y.shape)
print('Test data shape:  ', test_x.shape, test_y.shape)

class_names = [str(x) for x in range(10)]
class_count = len(class_names)
print('Class count:', class_count, class_names)

## I bet that you already remember the data 🙂

In [ ]:
show_example(train_x, train_y, class_names)

## Extract single image shape

In [ ]:
input_shape = train_x.shape[1:]
input_shape

## The autoencoder definition is really straightforward
* I hope that you would be able to define the architecture using the knowledge from previous lectures yourselves
    * 🔎 Why do we use `padding='same'`?
        * 💡 The convolutional layer will add padding to the input such that the output has the same width and height as the input
    * 🔎 Why do we have 128 numbers in the latent vector?
    * What does the ``UpSampling2D`` layer?
        * What interpolation methods do you know?

In [ ]:
autoencoder = keras.Sequential([
    keras.layers.Conv2D(64, (3,3), activation='relu', padding='same', input_shape=input_shape),
    keras.layers.MaxPooling2D((2, 2), padding='same'),
    keras.layers.Conv2D(32, (3,3), padding='same', activation='relu'),
    keras.layers.MaxPooling2D((2, 2), padding='same'),
    keras.layers.Conv2D(8, (3,3), padding='same', activation='relu'),    
    keras.layers.MaxPooling2D((2, 2), padding='same'),
    # a 128 values of the minimized knowledge / features
    keras.layers.Conv2D(8, (3,3), padding='same', activation='relu'),
    keras.layers.UpSampling2D((2,2)),
    keras.layers.Conv2D(32, (3,3), padding='same', activation='relu'),
    keras.layers.UpSampling2D((2,2)),
    keras.layers.Conv2D(64, (3,3), activation='relu'),
    keras.layers.UpSampling2D((2,2)),
    
    keras.layers.Conv2D(1, (3,3), activation='sigmoid', padding='same')
])

autoencoder.summary()

autoencoder.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

## Fit the model

In [ ]:
history = autoencoder.fit(train_x, train_x, validation_data=(valid_x, valid_x), epochs=10, batch_size=128)

show_history(history)

# Now you can generate original and reconstructed images
* ⚡ You feed-forward the test image set and the model will provide the reconstructed image on the output
* 🔎 How good is the reconstruction (just visually 🙂)?
    * On what parameter does the quality depend the most? 
    * 🔎 Do you think the output is blurry?
        * Why?
        * Do you remember the BCE loss function caveat? 
            * 💡 Take a look at the white pixels brightness 🙂

In [ ]:
predicted = autoencoder.predict(test_x)

n = 10
plt.figure(figsize=(20, 4))
for i in range(n):
    # display original
    ax = plt.subplot(2, n, i+1)
    plt.imshow(test_x[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(2, n, i + 1+n)
    plt.imshow(predicted[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

## We can also vizualize the encoded vectors
* We will just re-construct the encoder architecture and feed it with our pre-trained weights

In [ ]:
encoder = keras.Sequential([
    keras.layers.Conv2D(64, (3,3), activation='relu', padding='same', input_shape=(28,28,1)),
    keras.layers.MaxPooling2D((2, 2), padding='same'),
    keras.layers.Conv2D(32, (3,3), padding='same', activation='relu'),
    keras.layers.MaxPooling2D((2, 2), padding='same'),
    keras.layers.Conv2D(8, (3,3), padding='same', activation='relu'),    
    keras.layers.MaxPooling2D((2, 2), padding='same'),
])

# encoder.compile(optimizer='adam', loss='binary_crossentropy')
encoder.set_weights(autoencoder.get_weights()[:6])

for layer in encoder.layers:
    layer.trainable = False

encoder.summary()

## ⚡ Take a look at the latent vectors
* Do you see any differences among the vectors?
    * 💡 Compare vectors of *4* to other *4*s and then compare vectors of *4*s to vectors of *1*

In [ ]:
encoded = encoder.predict(test_x)

n = 10
plt.figure(figsize=(20, 8))
for i in range(n):
    ax = plt.subplot(2, n, i+1)
    plt.imshow(encoded[i].reshape(8, 16).T)
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    
    ax = plt.subplot(2, n, n+i+1)
    plt.imshow(test_x[i].reshape(28,28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

## Hey, but the classifier also has the latent vector after the encoder (CNN layers) part! 🤔
* The latent vector generated using the autoencoder principle generate a compressed representation of the input.
* Both models follow different set of goals and the goal of the autoencoder is different than from the classifier
    * 💡 Autoencoder want to compress the image into the latent vector in a way that it can be reconstructed with the best quality
    * 💡 Classifier want to embed the image into a latent vector as well, but the main goal is to create such representation that the vectors are different from each other so the model can distinguish among the images easily
    * 🔎 Which setting of the model has the biggest impact on the criterion?

* 💡 Some variants of encoder are able to generate better representation with some features - a sparse autoencoders that generate sparse representation for example.
    * 🔎 What is sparcity? Why would anyone want such property?

### Can we use latent vectors from just the **encoder** part of the autoencoder as a *feature engineering* step for classifier? 🤔
* 💡 Yes, it is possible but given the different properties of the latent vectors, there would be a negative accuracy impact
* **TL;DR: Use classifiers for classification tasks and autoencoder for autoencoder tasks**





# Outlier Detection With Autoencoders

> ### 📌 If you cannot compress it, there is probably something wrong with it

* A classical obstacle you stumble upon in data science and machine learning are the outliers
* The concept of an outlier is intuitively clear to a human, yet there is no generally meaningful mathematical definition apart from simple hacks that involve the standard deviation or the interquartile range

* 💡 **By definition, outliers are not like other data points.**

![Meme01](https://github.com/rasvob/VSB-FEI-Deep-Learning-Exercises/blob/main/images/dl_05_meme_01.png?raw=true)

* 🔎 Can you give some examples of an outlier in the data?

## How can we use autoencoders for the outlier detection?
* 💡 A good autoencoder should be able to compress (encode) data into fewer dimensions and then decompress (decode) it again without introducing many errors
    * An autoencoder tries to learn good encodings for a given dataset
    * 💡 Since most data points in the dataset are not outliers, the autoencoder will be influenced the most by "normal" data
* **📌 If the autoencoder introduces a large error for some data point, this data point might be an outlier**

## ⚡ We can test it out
* First, create sample dataset of 25 images

In [ ]:
sample_x = test_x[:25]
sample_y = test_y[:25]

show_example(sample_x, sample_y, class_names)

## Now we will add a noise in the data to create the **outliers**

In [ ]:
modified_x = np.clip(sample_x + 0.1 * np.random.normal(loc=0.0, scale=1.0, size=sample_x.shape), 0., 1.) 
modified_y = sample_y
show_example(modified_x, modified_y, class_names)

## The first step is to create the reconstructed representation for the **original** samples

In [ ]:
sample_predicted = autoencoder.predict(sample_x)
show_example(sample_predicted, sample_y, class_names)

## Then we will create the reconstructed representation for the **noisy** samples

In [ ]:
modified_pred = autoencoder.predict(modified_x)
show_example(modified_pred, modified_y, class_names)

## 🚀 Now we can compute the reconstruction errors for both of the image sets using Frobenius norm
* https://numpy.org/doc/stable/reference/generated/numpy.linalg.norm.html
* Frobenius norm (Euclidean norm) ~ Root of squared errors

In [ ]:
sample_norms = [np.linalg.norm(sample_x[i].reshape(28,28) - sample_predicted[i].reshape(28,28)) for i in range(len(sample_x))]
modified_norms = [np.linalg.norm(modified_x[i].reshape(28,28) - modified_pred[i].reshape(28,28)) for i in range(len(sample_x))]

sample_mean, sample_std = np.mean(sample_norms), np.std(sample_norms)
modified_mean, modified_std = np.mean(modified_norms), np.std(modified_norms)

sample_mean, modified_mean

## The final step is the visualization of the errors distribution
* 🔎 Can you see an outlier data in the plot?
    * How would you detect them?

In [ ]:
plt.figure(figsize=(16,9))
xmin = np.min([sample_mean-4*sample_std, modified_mean-4*modified_std])
xmax = np.max([sample_mean+4*sample_std, modified_mean+4*modified_std])

x = np.linspace(xmin, xmax, 100)

y = scipy.stats.norm.pdf(x,sample_mean,sample_std)

plt.plot(x,y)

y = scipy.stats.norm.pdf(x,modified_mean,modified_std)

plt.plot(x,y)

plt.bar(x=sample_norms, height=[0.6 for x in sample_norms], width=0.01)
plt.bar(x=modified_norms, height=[0.6 for x in sample_norms], width=0.01)
plt.show()

# 📒 Variational Autoencoder (VAE)
* A variational autoencoder (VAE) provides a probabilistic manner for describing an observation in latent space

* Rather than building an encoder which outputs a single value to describe each latent state attribute, we'll formulate our encoder to describe a probability distribution for each latent attribute

* 💡 Let's suppose we've trained an autoencoder model on a large dataset of faces with a encoding dimension of 6
    * An ideal autoencoder will learn descriptive attributes of faces such as skin color, whether or not the person is wearing glasses, etc. in an attempt to describe an observation in some compressed representation

![VAE01](https://github.com/rasvob/VSB-FEI-Deep-Learning-Exercises/blob/main/images/dl_05_vae_01.png?raw=true)

* 📌 **Using a variational autoencoder, we can describe latent attributes in probabilistic terms**

![VAE02](https://github.com/rasvob/VSB-FEI-Deep-Learning-Exercises/blob/main/images/dl_05_vae_02.png?raw=true)

![VAE03](https://github.com/rasvob/VSB-FEI-Deep-Learning-Exercises/blob/main/images/dl_05_vae_03.png?raw=true)

* By constructing our encoder model to output a range of possible values (a statistical distribution) from which we'll randomly sample to feed into our decoder model, we're essentially enforcing a continuous, smooth latent space representation 
    * 💡 For any sampling of the latent distributions, we're expecting our decoder model to be able to accurately reconstruct the input
    * 📌 Thus, values which are nearby to one another in latent space should correspond with very similar reconstructions

![VAE04](https://github.com/rasvob/VSB-FEI-Deep-Learning-Exercises/blob/main/images/dl_05_vae_04.png?raw=true)


* **You can see [this](https://www.jeremyjordan.me/variational-autoencoders/) for very good explanation**

## 🚀 Let's try it out!

In [ ]:
# mnist is the basic dataset for image classifaction
dataset = tf.keras.datasets.mnist

# data from any dataset are loaded using the load_Data function
(train_x, train_y), (test_x, test_y) = dataset.load_data()

train_x = train_x.reshape(*train_x.shape, 1)
test_x = test_x.reshape(*test_x.shape, 1)

train_x = train_x/255.0
test_x = test_x/255.0

train_x, valid_x, train_y, valid_y = train_test_split(train_x, train_y, test_size=0.2, random_state=42)

# the data are in the form of 28x28 pixes with values 0-255.
print('Train data shape: ', train_x.shape, train_y.shape)
print('Validation data shape: ', valid_x.shape, valid_y.shape)
print('Test data shape:  ', test_x.shape, test_y.shape)

class_names = [str(x) for x in range(10)]
class_count = len(class_names)
print('Class count:', class_count, class_names)

## ⚠ There are some differences to the traditional classification task approach

* We will reshape data from 28x28 to 32x32x1 shape
    * We will use subsampling/pooling in the model so we expect division of width and height by 2 with each of the subsampling step
    * ⚠ Dividing 28 by 2 multiple times will lead to sequence -> `28 >> 14 >> 7 >> ...` 
        * Number 7 could be an issue for reconstruction because the subsampling uses integer division (7 / 2 = 3) but 2*3 = 6

* 💡 Resizing to 32x32 will save us from this issue because we will get sequence `32 >> 16 >> 8 >> 4 >> 2 >> 1` which is no problem to reconstruct

In [ ]:
size = 32

train_x_res = np.array([cv2.resize(x, (size, size)) for x in train_x])
test_x_res = np.array([cv2.resize(x, (size, size)) for x in test_x])
valid_x_res = np.array([cv2.resize(x, (size, size)) for x in valid_x])

train_x_r = train_x_res.reshape(-1, size, size, 1).astype('float32')
test_x_r = test_x_res.reshape(-1, size, size, 1).astype('float32')
valid_x_r = valid_x_res.reshape(-1, size, size, 1).astype('float32')

## Check the shapes

In [ ]:
train_x_r.shape, test_x_res.shape, valid_x_res.shape

In [ ]:
input_shape = train_x_r.shape[1:]
input_shape

#### Just for the compatibility of TF versions

In [ ]:
from tensorflow.python.framework.ops import disable_eager_execution
disable_eager_execution()

# Set the latent vector dimension first
* Latent dimension define frow how many distributions we want to sample
* 💡 Each distribution a is Gaussian one, so we are creating basically multivariate Gaussian distribution

In [ ]:
latent_dim = 2

## The last two Dense layers represent vectors of mean and logarithm of variance of the individual distributions
* Sampling function will combine these vectors with samples from Normal (Gaussian) distribution ~ N(0, 1)

In [ ]:
def sampling(args):
    mean_mu, log_var = args
    epsilon = keras.backend.random_normal(shape=keras.backend.shape(mean_mu), mean=0., stddev=1.)
    return mean_mu + keras.backend.exp(log_var/2)*epsilon

## 💡 It's worth to mention that we use strides=2 in the Conv2D layers instead of strides=1 so we are doing subsampling, beside the filtering, in this layers

In [ ]:
encoder_input = keras.layers.Input(shape=input_shape)
x = keras.layers.Conv2D(28, (3,3), strides=2, padding='same', activation='relu')(encoder_input)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Conv2D(64, (3,3), strides=2, padding='same', activation='relu')(x)
x = keras.layers.BatchNormalization()(x)

before_flatten_shape = keras.backend.int_shape(x)[1:]
print(before_flatten_shape)

x = keras.layers.Flatten()(x)
mean_mu = keras.layers.Dense(latent_dim, name='mean_mu')(x)
log_var = keras.layers.Dense(latent_dim, name='log_var')(x)
encoder_output = keras.layers.Lambda(sampling)([mean_mu, log_var])

enc_model = keras.Model(encoder_input, encoder_output)
enc_model.summary()

## We can take a look at the model

In [ ]:
tf.keras.utils.plot_model(enc_model)

# ⚡ Now we need to define the decoder part
* Input shape is equal to the latent dimensions count
* We use Conv2DTranspose, sometimes called de-conv layer
* A simple way of how to think about it is that it both performs the upsample operation and interprets the coarse input data to fill in the detail while it is upsampling
    * 💡 It is like a layer that combines the UpSampling2D and Conv2D layers into one layer
    * Models that use these layers can be referred to as deconvolutional networks, or deconvnets
* 📌 Unlike upsampling, it involves trainable parameters and **can learn to add new information** while increasing the size of the input

* A deconvnet can be thought of as a convnet model that uses the same components (filtering, pooling) but in reverse, so **instead of mapping pixels to features it does the opposite**

### strides=2 in the Conv2DTranspose is responsible for the upsampling operation
* The Reshape layers creates from the vector of 64 elements matrix 8x8 which can be upsampled to the original size

In [ ]:
decoder_input = keras.layers.Input(shape = (latent_dim,) , name = 'decoder_input')
x = keras.layers.Dense(np.prod(before_flatten_shape))(decoder_input)
x = keras.layers.Reshape(before_flatten_shape)(x)
x = keras.layers.Conv2DTranspose(128, (3,3), strides=2, padding='same', activation='relu')(x)
x = keras.layers.Conv2DTranspose(1, (3,3), strides=2, padding='same', activation='sigmoid')(x)
decoder_output = x

dec_model = keras.Model(decoder_input, decoder_output)
dec_model.summary()

## ⚡ Now we will just connect the two parts together


In [ ]:
vae_input = encoder_input

vae_output = dec_model(encoder_output)

vae_model = keras.Model(vae_input, vae_output)

## 💡 VAE uses specific loss function which we have to define ourselves
* The loss function combines RMSE with KL-divergence
    * KL divergence score, quantifies how much one probability distribution differs from another probability distribution.
        * https://machinelearningmastery.com/divergence-between-probability-distributions/
        * https://towardsdatascience.com/light-on-math-machine-learning-intuitive-guide-to-understanding-kl-divergence-2b382ca2b2a8

* We use the KL-divergence score to penalize the activation values of the mean and log_variance layers if they differ from N(0, 1) distribution - 
    * We want to be as close as possible to this distribution
    * 🔎 Why?

In [ ]:
def total_loss_closure(mean_mu, log_var):
    def r_loss(y_true, y_pred):
        return keras.backend.mean(keras.backend.square(y_true - y_pred), axis=[0, 1])

    def kl_loss(y_true, y_pred):
        kl_loss =  -0.5 * keras.backend.sum(1 + log_var - keras.backend.square(mean_mu) - keras.backend.exp(log_var), axis = 1)
        return kl_loss

    def loss(y_true, y_pred):
        LOSS_FACTOR = 10000
        return LOSS_FACTOR*r_loss(y_true, y_pred) + kl_loss(y_true, y_pred)

    return loss

In [ ]:
vae_model.compile(optimizer='adam', loss = total_loss_closure(mean_mu, log_var))

In [ ]:
batch_size = 128
epochs = 10
history = vae_model.fit(train_x_r, train_x_r, validation_data=(valid_x_r, valid_x_r), epochs=epochs, batch_size=batch_size)

## How will the reconstructed images look like?
* 🚀 Let's take a look!

In [ ]:
predicted = vae_model.predict(test_x_r)

n = 10
plt.figure(figsize=(20, 4))
for i in range(n):
    # display original
    ax = plt.subplot(2, n, i+1)
    plt.imshow(test_x_r[i].reshape(32, 32))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(2, n, i + 1+n)
    plt.imshow(predicted[i].reshape(32, 32))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

## How can we use VAE for creating new images? 🤔
* If we use just the decoder part of the model and feed it samples from distribution N(0, 1) we can generate new images with the model!

In [ ]:
n = 10
reconst_images = dec_model.predict(np.random.normal(0,1,size=(n,latent_dim)))

plt.figure(figsize=(20, 4))
for i in range(n):
    # display original
    ax = plt.subplot(2, n, i+1)
    plt.imshow(reconst_images[i].reshape(32, 32))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

## 📈 We can display even the whole manifold
* 🔎 How can we select scale?

* 💡 3-sigma rule is helpful!
   * 𝑃(𝜇 − 𝑘𝜎 < 𝑋 < 𝜇 + 𝑘𝜎) = 0.954 for k=2
   * k = 2, 𝜇 = 0, 𝜎 = 1 in our case so 95,4% of numbers will be in interval <-2, 2>


In [ ]:
def plot_latent(decoder):
    # display a n*n 2D manifold of digits
    n = 30
    digit_size = 32
    scale = 2.0
    figsize = 15
    figure = np.zeros((digit_size * n, digit_size * n))
    # linearly spaced coordinates corresponding to the 2D plot
    # of digit classes in the latent space
    grid_x = np.linspace(-scale, scale, n)
    grid_y = np.linspace(-scale, scale, n)[::-1]

    for i, yi in enumerate(grid_y):
        for j, xi in enumerate(grid_x):
            z_sample = np.array([[xi, yi]])
            x_decoded = decoder.predict(z_sample)
            digit = x_decoded[0].reshape(digit_size, digit_size)
            figure[
                i * digit_size : (i + 1) * digit_size,
                j * digit_size : (j + 1) * digit_size,
            ] = digit

    plt.figure(figsize=(figsize, figsize))
    start_range = digit_size // 2
    end_range = n * digit_size + start_range + 1
    pixel_range = np.arange(start_range, end_range, digit_size)
    sample_range_x = np.round(grid_x, 1)
    sample_range_y = np.round(grid_y, 1)
#     plt.xticks(pixel_range, sample_range_x)
#     plt.yticks(pixel_range, sample_range_y)
#     plt.xlabel("z[0]")
#     plt.ylabel("z[1]")
    plt.imshow(figure, cmap="Greys_r")
    plt.show()

plot_latent(dec_model)

# ✅  Tasks for the lecture (2p)

* Implement an autoencoder model that is able to denoise an image
    * 💡 Implement it on the data defined below

* The denoising autoencoder is an autoencoder that will learn how to remove random noise from the images
    * First, noisy images have to be generated
    * Then the autoencoder need to be defined and trained
    * In the end, denoised images may be reconstructed
    
* Experiment with **noise_factor** values of `0.1`, `0.5` and `0.9` 
    * 📒Write down a short summary about the reconstruction capabilities of the model

In [ ]:
noise_factor = 0.5
noisy_train_x = np.clip(train_x + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=train_x.shape), 0., 1.)
noisy_valid_x = np.clip(valid_x + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=valid_x.shape), 0., 1.)
noisy_test_x = np.clip(test_x + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=test_x.shape), 0., 1.) 


n = 10
plt.figure(figsize=(2*n, 4))
for i in range(n):
    ax = plt.subplot(2, n, i+1)
    plt.imshow(train_x[i].reshape(28,28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    
    ax = plt.subplot(2, n, n+i+1)
    plt.imshow(noisy_train_x[i].reshape(28,28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()